In [70]:
#this script checks missing and present data for participants
#this is currently set up for the AOL study Qualtrics measures
#
#TO USE:
#download .csv files with labels and extract the zipped file
#put the unzipped file in the data_dir directory
#update the participants you need to check
#save the file to whatever directory works best for you
# 
# BP 8/7/20

#SET UP#

#calling libraries
import pandas as pd
import xlwt
from xlwt import Workbook
from os import listdir
import os
from os.path import isfile, join
from zipfile import ZipFile 
import glob 
from datetime import datetime

#date the data was downloaded (may not be the day this script is run)
data_date="2020-08-14"

#list of updated subs you want to check
#script will not run if you try to add a subject that does not exist on redcap
subs_filename = "current_subs.csv" 

#directory where the current_subs file is stored. Must have double \\ not just a single \. Must have \\ at end of pathway
sub_dir= 'D:\\Data Backup\\data checks\\'
#directory where you want to save the check output files. Must have double \\ not just a single \. Must have \\ at end of pathway
save_dir = 'D:\\Data Backup\\data checks\\output\\'
#directory where you will temporarily store unzipped files; not permanent and will detele unzips to save space
unzip_temp_dir = 'D:\\Data Backup\\data checks\\unzipped\\'
#directories where data is stored. Must have double \\ not just a single \. Must have \\ at end of pathway
qualtrics_dir = 'D:\\Data Backup\\Qualtrics Data\\'
redcap_dir = 'D:\\Data Backup\\REDCap Data\\'

sv_icf_dir = qualtrics_dir + data_date + '\\1. SV ICFs\\'
audio_icf_dir = qualtrics_dir + data_date + '\\2. Audio ICF\\'
informant_icf_dir = qualtrics_dir + data_date + '\\3. Informant ICF\\'
sigt_dir = qualtrics_dir + data_date + '\\4. SIGT player preference\\'
face_a_dir = qualtrics_dir + data_date + '\\6. FACE task A\\'
face_b_dir = qualtrics_dir + data_date + '\\7. FACE task B\\'
video_a_dir = qualtrics_dir + data_date + '\\8. Lie video task A\\'
video_b_dir = qualtrics_dir + data_date + '\\9. Lie video task B\\'
picture_dir = qualtrics_dir + data_date + '\\10. Picture Task\\'
vocab_dir = qualtrics_dir + data_date + '\\11. Shipley Vocab\\'
fnt_dir = qualtrics_dir + data_date + '\\12. FNT\\'
temporal_gains_dir = qualtrics_dir + data_date + '\\13. Temporal Discounting gains\\'
temporal_losses_dir = qualtrics_dir + data_date + '\\14. Temporal Discounting losses\\'

preassessment_dir = qualtrics_dir + data_date + '\\5. Pre assessment\\'
assessment_1_dir = qualtrics_dir + data_date + '\\15. Assessment 1\\'
assessment_2_dir = qualtrics_dir + data_date + '\\16. Assessment 2\\'
assessment_3_dir = qualtrics_dir + data_date + '\\17. Assessment 3\\'
assessment_4_dir = qualtrics_dir + data_date + '\\18. Assessment 4\\'
assessment_5_dir = qualtrics_dir + data_date + '\\19. Assessment 5\\'
assessment_6_dir = qualtrics_dir + data_date + '\\20. Assessment 6\\'
assessment_7_dir = qualtrics_dir + data_date + '\\21. Assessment 7\\'
assessment_8_dir = qualtrics_dir + data_date + '\\22. Assessment 8\\'
#uncomment the following when you need to add them
#assessment_9_dir = qualtrics_dir + data_date + '\\23. Assessment 9\\'
#assessment_10_dir = qualtrics_dir + data_date + '\\24. Assessment 10\\'
#assessment_11_dir = qualtrics_dir + data_date + '\\25. Assessment 11\\'
#assessment_12_dir = qualtrics_dir + data_date + '\\26. Assessment 12\\'

#selecting the most recent redcap file
#listing all files in redcap_dir
all_redcap_files = [f for f in listdir(redcap_dir) if isfile(join(redcap_dir, f))]
#sorting list of files
all_redcap_files.sort(reverse=True)
#selecting top file (most recent date)
redcap_file=all_redcap_files[0]

#session names in redcap checklists -- to see if session has been completed yet
sv = "screening_visit_remote_experimenter_checklist_complete"
follow_up_1 = "followup_visit_1_remote_experimenter_checklist_complete"
follow_up_2 = "followup_visit_2_remote_experimenter_checklist_complete"
physio = "physio_visit_on_campus_experimenter_checklist_complete"
MRI ="mri_visit_on_campus_experimenter_checklist_complete"
scd_participant = "scd_visit_participant_remote_experimenter_checklis_complete"
scd_informant ="scd_visit_informant_remote_experimenter_checklist_complete"

#file names to save output. Update with date of most recent data (match the datafile)
save_name_present = 'qualtrics_present_' + data_date + '.xls'
save_name_missing = 'qualtrics_missing_' + data_date + '.xls'

#opening 2 workbooks to write results
wb1 = Workbook()
wb2 = Workbook()

#reading in the subs file
col_list = ["subIDs"]
df = pd.read_csv(r'' + sub_dir + subs_filename, usecols=col_list) #read in file
df["subIDs"] = df["subIDs"].apply(str)

#turning excel column into list so we can manipulate the data
df_updating_subs = df["subIDs"].tolist()

In [71]:
#1. SV ICF
#counts number of ICFs present in Qualtrics and compares to the number of participants who have completed their SV session according to redcap
#printed at end of cell. No excel sheet made for this.

measure_dir = sv_icf_dir
progress_column = "Progress" #only selecting items that are 100% complete as listed in the Progress column
session = sv

#SELECTING QUALTRICS FILE
#listing files in measure_dir and selecting top option (there should only be 1 file in each dir -- if not, there could be issues)
dir_file = [f for f in listdir(measure_dir) if isfile(join(measure_dir, f))]
dir_file = dir_file[0]
#extracting the top file from measure_dir and putting into zip_temp_dir
with ZipFile(measure_dir + dir_file, 'r') as zip_ref:
    zip_ref.extractall(unzip_temp_dir)
#listing files in unzip_temp_dir (there should only be 1 file in each dir -- if not, there could be issues)
filename = [f for f in listdir(unzip_temp_dir) if isfile(join(unzip_temp_dir, f))]
#selecting file
filename = filename[0]

#READING IN QUALTRICS FILE
#creating a list of 1 header name. Only this 1 column will be read in
col_list_qual = [progress_column]
#reading in the qualtrics file with only the header specified
df_qual = pd.read_csv(r'' + unzip_temp_dir + filename,  usecols=col_list_qual)

#READING IN REDCAP FILE
#reading in the redcap file -- this is just to check if session has been completed
col_list_redcap = [session]
df_redcap = pd.read_csv(r'' + redcap_dir + redcap_file,  usecols=col_list_redcap)

#turning the imported excel columns into lists so we can manipulate the data
df_qual_progress = df_qual[progress_column].tolist()
df_red_ses_check = df_redcap[session].tolist()

#counting nummber of SV ICFs in qualtrics that are 100 percent complete
counter_ICF_complete=0
for x in df_qual_progress:
    if x == "100":
        counter_ICF_complete = counter_ICF_complete + 1
        
#counting nummber of complete SV sessions as listed in redcap
counter_session_complete=0
for x in df_red_ses_check:
    if x == 2:
        counter_session_complete = counter_session_complete + 1
    
#counting length of df_updating_subs (which is a list from current_subs.csv) and number of ICFs listed at 100% complete in Qualtrics
if counter_ICF_complete == counter_session_complete:
    print("Correct number of SV ICFs")
if len(df_updating_subs) != counter_complete:
    print("There are " + str(counter_session_complete) + " subjects who have completed their SV session according to redcap")
    print("But there are " + str(counter_ICF_complete) + " SV ICFs listed in qualtrics as complete")
    
os.remove(unzip_temp_dir + filename)

There are 42 subjects who have completed their SV session according to redcap
But there are 47 SV ICFs listed in qualtrics as complete


In [72]:
#2. Audio ICF
#counts number of ICFs present in Qualtrics and compares to the number of participants who have completed their SV session according to redcap
#printed at end of cell. No excel sheet made for this.

measure_dir = audio_icf_dir
progress_column = "Progress" #only selecting items that are 100% complete as listed in the Progress column
session = follow_up_2

#SELECTING QUALTRICS FILE
#listing files in measure_dir and selecting top option (there should only be 1 file in each dir -- if not, there could be issues)
dir_file = [f for f in listdir(measure_dir) if isfile(join(measure_dir, f))]
dir_file = dir_file[0]
#extracting the top file from measure_dir and putting into zip_temp_dir
with ZipFile(measure_dir + dir_file, 'r') as zip_ref:
    zip_ref.extractall(unzip_temp_dir)
#listing files in unzip_temp_dir (there should only be 1 file in each dir -- if not, there could be issues)
filename = [f for f in listdir(unzip_temp_dir) if isfile(join(unzip_temp_dir, f))]
#selecting file
filename = filename[0]

#READING IN QUALTRICS FILE
#creating a list of 1 header name. Only this 1 column will be read in
col_list_qual = [progress_column]
#reading in the qualtrics file with only the header specified
df_qual = pd.read_csv(r'' + unzip_temp_dir + filename,  usecols=col_list_qual)

#READING IN REDCAP FILE
#reading in the redcap file -- this is just to check if session has been completed
col_list_redcap = [session]
df_redcap = pd.read_csv(r'' + redcap_dir + redcap_file,  usecols=col_list_redcap)

#turning the imported excel columns into lists so we can manipulate the data
df_qual_progress = df_qual[progress_column].tolist()
df_red_ses_check = df_redcap[session].tolist()

#counting nummber of SV ICFs in qualtrics that are 100 percent complete
counter_ICF_complete=0
for x in df_qual_progress:
    if x == "100":
        counter_ICF_complete = counter_ICF_complete + 1
        
#counting nummber of complete SV sessions as listed in redcap
counter_session_complete=0
for x in df_red_ses_check:
    if x == 2:
        counter_session_complete = counter_session_complete + 1
    
#counting length of df_updating_subs (which is a list from current_subs.csv) and number of ICFs listed at 100% complete in Qualtrics
if counter_ICF_complete == counter_session_complete:
    print("Correct number of audio ICFs")
if len(df_updating_subs) != counter_complete:
    print("There are " + str(counter_session_complete) + " subjects who have completed their follow up 2 session according to redcap")
    print("But there are " + str(counter_ICF_complete) + " audio ICFs listed in qualtrics as complete")
    
os.remove(unzip_temp_dir + filename)

There are 18 subjects who have completed their follow up 2 session according to redcap
But there are 34 audio ICFs listed in qualtrics as complete


In [73]:
#3. Informant ICF
#counts number of ICFs present in Qualtrics and compares to the number of participants who have completed their SV session according to redcap
#printed at end of cell. No excel sheet made for this.

measure_dir = informant_icf_dir
progress_column = "Progress" #only selecting items that are 100% complete as listed in the Progress column
session = scd_informant

#SELECTING QUALTRICS FILE
#listing files in measure_dir and selecting top option (there should only be 1 file in each dir -- if not, there could be issues)
dir_file = [f for f in listdir(measure_dir) if isfile(join(measure_dir, f))]
dir_file = dir_file[0]
#extracting the top file from measure_dir and putting into zip_temp_dir
with ZipFile(measure_dir + dir_file, 'r') as zip_ref:
    zip_ref.extractall(unzip_temp_dir)
#listing files in unzip_temp_dir (there should only be 1 file in each dir -- if not, there could be issues)
filename = [f for f in listdir(unzip_temp_dir) if isfile(join(unzip_temp_dir, f))]
#selecting file
filename = filename[0]

#READING IN QUALTRICS FILE
#creating a list of 1 header name. Only this 1 column will be read in
col_list_qual = [progress_column]
#reading in the qualtrics file with only the header specified
df_qual = pd.read_csv(r'' + unzip_temp_dir + filename,  usecols=col_list_qual)

#READING IN REDCAP FILE
#reading in the redcap file -- this is to check if session has been completed
col_list_redcap = [session]
df_redcap = pd.read_csv(r'' + redcap_dir + redcap_file,  usecols=col_list_redcap)

#turning the imported excel columns into lists so we can manipulate the data
df_qual_progress = df_qual[progress_column].tolist()
df_red_ses_check = df_redcap[session].tolist()

#counting nummber of SV ICFs in qualtrics that are 100 percent complete
counter_ICF_complete=0
for x in df_qual_progress:
    if x == "100":
        counter_ICF_complete = counter_ICF_complete + 1
        
#counting nummber of complete SV sessions as listed in redcap
counter_session_complete=0
for x in df_red_ses_check:
    if x == 2:
        counter_session_complete = counter_session_complete + 1
    
#counting length of df_updating_subs (which is a list from current_subs.csv) and number of ICFs listed at 100% complete in Qualtrics
if counter_ICF_complete == counter_session_complete:
    print("Correct number of audio ICFs")
if len(df_updating_subs) != counter_complete:
    print("There are " + str(counter_session_complete) + " subjects who have completed the SCD informant session according to redcap")
    print("But there are " + str(counter_ICF_complete) + " informant ICFs listed in qualtrics as complete")
    
os.remove(unzip_temp_dir + filename)

There are 1 subjects who have completed the SCD informant session according to redcap
But there are 2 informant ICFs listed in qualtrics as complete


In [74]:
#4. SIGT player preference

#setting dir for this measure and identifying subID column in this qualtrics file
measure_dir = sigt_dir
id_column = "Q65"
session = physio

#creating the sheets in the excel file where output will be written
sheet1 = wb1.add_sheet('SIGT present')
sheet2 = wb2.add_sheet('SIGT missing')

#putting headers of subID in each excel sheet
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

#SELECTING QUALTRICS FILE
#listing files in measure_dir and selecting top option (there should only be 1 file in each dir -- if not, there could be issues)
dir_file = [f for f in listdir(measure_dir) if isfile(join(measure_dir, f))]
dir_file = dir_file[0]
#extracting the top file from measure_dir and putting into zip_temp_dir
with ZipFile(measure_dir + dir_file, 'r') as zip_ref:
    zip_ref.extractall(unzip_temp_dir)
#listing files in unzip_temp_dir (there should only be 1 file in each dir -- if not, there could be issues)
filename = [f for f in listdir(unzip_temp_dir) if isfile(join(unzip_temp_dir, f))]
#selecting file
filename = filename[0]

#READING IN QUALTRICS FILE
#creating a list of 1 header name. Only this 1 column will be read in
col_list_qual = [id_column]
#reading in the qualtrics file with only the header specified
df_qual = pd.read_csv(r'' + unzip_temp_dir + filename,  usecols=col_list_qual)

#READING IN REDCAP FILE
#reading in the redcap file -- this is just for SIGT to check if SIGT was needed
col_list_redcap = ["record_id", "v5_sigt_alreadydone", session, "aol_phit_id", "aol_task_id"]
df_redcap = pd.read_csv(r'' + redcap_dir + redcap_file,  usecols=col_list_redcap)

#turning the imported excel columns into lists so we can manipulate the data
df_qual_id = df_qual[id_column].tolist()
df_red_id = df_redcap["record_id"].tolist()
df_red_ses_check = df_redcap[session].tolist()
df_red_sigt_done = df_redcap["v5_sigt_alreadydone"].tolist()
df_phit_id = df_redcap["aol_phit_id"].tolist()
df_task_id = df_redcap["aol_task_id"].tolist()


for x in df_updating_subs:
    loc = df_red_id.index(x) #finding location of sub in redcap file
    if df_red_sigt_done[loc]==0: #checking if SIGT has already been done. if 0, has not been done, and needs to be done
        if df_red_ses_check[loc]==2: #checking if session has been completed
            if x in df_qual_id: #checking if AOL ID is in qualtrics measure; if so, writing to excel present file
                place_counter_complete=place_counter_complete+1 #updating the present place counter so we will write to a new cell
                sheet1.write(place_counter_complete, 0, x) #write the AOL subID (x) into the correct cell in the complete workbook
            if x not in df_qual_id: #if AOL ID is not in qualtrics measure
                if df_phit_id[loc] in df_qual_id: #checking if PHIT ID is in qualtrics measure
                    place_counter_complete=place_counter_complete+1 
                    sheet1.write(place_counter_complete, 0, x) 
                if df_phit_id[loc] not in df_qual_id: #if PHIT ID is not in qualtrics measure
                    if df_task_id[loc] in df_qual_id: #checking if TASK ID is in qualtrics measure
                        place_counter_complete=place_counter_complete+1 
                        sheet1.write(place_counter_complete, 0, x) 
                    if df_task_id[loc] not in df_qual_id: #if TASK ID is not in qualtrics, no IDs are in qualtrics, so this is missing
                        place_counter_missing=place_counter_missing+1 #updating the missing place counter so we will write to a new cell
                        sheet2.write(place_counter_missing, 0, x) #write the subID (x) into the correct cell in the missing workbook
        
#saving the workborks .xlsx files        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

os.remove(unzip_temp_dir + filename)

In [75]:
#5. Pre assessment

#setting dir for this measure and identifying subID column in this qualtrics file
measure_dir = preassessment_dir
id_column = "ID"
session = SV

#creating the sheets in the excel file where output will be written
sheet1 = wb1.add_sheet('Preassessment present')
sheet2 = wb2.add_sheet('Preassessment missing')

#putting headers of subID in each excel sheet
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

#SELECTING QUALTRICS FILE
#listing files in measure_dir and selecting top option (there should only be 1 file in each dir -- if not, there could be issues)
dir_file = [f for f in listdir(measure_dir) if isfile(join(measure_dir, f))]
dir_file = dir_file[0]
#extracting the top file from measure_dir and putting into zip_temp_dir
with ZipFile(measure_dir + dir_file, 'r') as zip_ref:
    zip_ref.extractall(unzip_temp_dir)
#listing files in unzip_temp_dir (there should only be 1 file in each dir -- if not, there could be issues)
filename = [f for f in listdir(unzip_temp_dir) if isfile(join(unzip_temp_dir, f))]
#selecting file
filename = filename[0]

#READING IN QUALTRICS FILE
#creating a list of 1 header name. Only this 1 column will be read in
col_list_qual = [id_column]
#reading in the qualtrics file with only the header specified
df_qual = pd.read_csv(r'' + unzip_temp_dir + filename,  usecols=col_list_qual)

#READING IN REDCAP FILE
#reading in the redcap file -- this is just for SIGT to check if SIGT was needed
col_list_redcap = ["record_id", session, "aol_phit_id", "aol_task_id"]
df_redcap = pd.read_csv(r'' + redcap_dir + redcap_file,  usecols=col_list_redcap)

#turning the imported excel columns into lists so we can manipulate the data
df_qual_id = df_qual[id_column].tolist()
df_red_id = df_redcap["record_id"].tolist()
df_red_ses_check = df_redcap[session].tolist()
df_phit_id = df_redcap["aol_phit_id"].tolist()
df_task_id = df_redcap["aol_task_id"].tolist()

                    
for x in df_updating_subs:
    loc = df_red_id.index(x) #finding location of sub in redcap file
    if df_red_ses_check[loc]==2: 
        if x in df_qual_id: 
            place_counter_complete=place_counter_complete+1 
            sheet1.write(place_counter_complete, 0, x) 
        if x not in df_qual_id: 
            if df_phit_id[loc] in df_qual_id: #checking if PHIT ID is in qualtrics measure
                place_counter_complete=place_counter_complete+1 
                sheet1.write(place_counter_complete, 0, x) 
            if df_phit_id[loc] not in df_qual_id: 
                if df_task_id[loc] in df_qual_id: #checking if TASK ID is in qualtrics measure
                    place_counter_missing=place_counter_missing+1
                    sheet2.write(place_counter_missing, 0, x)
                    
#saving the workborks .xlsx files        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

os.remove(unzip_temp_dir + filename)

In [76]:
#6. FACE task A

#setting dir for this measure and identifying subID column in this qualtrics file
measure_dir = face_a_dir
id_column = "ID"
session = follow_up_1

#creating the sheets in the excel file where output will be written
sheet1 = wb1.add_sheet('Face A present')
sheet2 = wb2.add_sheet('Face A missing')

#putting headers of subID in each excel sheet
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

#SELECTING QUALTRICS FILE
#listing files in measure_dir and selecting top option (there should only be 1 file in each dir -- if not, there could be issues)
dir_file = [f for f in listdir(measure_dir) if isfile(join(measure_dir, f))]
dir_file = dir_file[0]
#extracting the top file from measure_dir and putting into zip_temp_dir
with ZipFile(measure_dir + dir_file, 'r') as zip_ref:
    zip_ref.extractall(unzip_temp_dir)
#listing files in unzip_temp_dir (there should only be 1 file in each dir -- if not, there could be issues)
filename = [f for f in listdir(unzip_temp_dir) if isfile(join(unzip_temp_dir, f))]
#selecting file
filename = filename[0]

#READING IN QUALTRICS FILE
#creating a list of 1 header name. Only this 1 column will be read in
col_list_qual = [id_column]
#reading in the qualtrics file with only the header specified
df_qual = pd.read_csv(r'' + unzip_temp_dir + filename,  usecols=col_list_qual)

#READING IN REDCAP FILE
#reading in the redcap file -- this is just for SIGT to check if SIGT was needed
col_list_redcap = ["record_id", session, "aol_phit_id", "aol_task_id"]
df_redcap = pd.read_csv(r'' + redcap_dir + redcap_file,  usecols=col_list_redcap)

#turning the imported excel columns into lists so we can manipulate the data
df_qual_id = df_qual[id_column].tolist()
df_red_id = df_redcap["record_id"].tolist()
df_red_ses_check = df_redcap[session].tolist()
df_phit_id = df_redcap["aol_phit_id"].tolist()
df_task_id = df_redcap["aol_task_id"].tolist()
            
for x in df_updating_subs:
    if (int(x[5]) % 2) != 0:  #checking if last digit of subID is odd
        loc = df_red_id.index(x) #finding location of sub in redcap file
        if df_red_ses_check[loc]==2: 
            if x in df_qual_id: 
                place_counter_complete=place_counter_complete+1 
                sheet1.write(place_counter_complete, 0, x) 
            if x not in df_qual_id: 
                if df_phit_id[loc] in df_qual_id: #checking if PHIT ID is in qualtrics measure
                    place_counter_complete=place_counter_complete+1 
                    sheet1.write(place_counter_complete, 0, x) 
                if df_phit_id[loc] not in df_qual_id: 
                    if df_task_id[loc] in df_qual_id: #checking if TASK ID is in qualtrics measure
                        place_counter_missing=place_counter_missing+1
                        sheet2.write(place_counter_missing, 0, x)
        
#saving the workborks .xlsx files        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

os.remove(unzip_temp_dir + filename)

In [77]:
#7. FACE task B

#setting dir for this measure and identifying subID column in this qualtrics file
measure_dir = face_b_dir
id_column = "ID"
session = follow_up_1

#creating the sheets in the excel file where output will be written
sheet1 = wb1.add_sheet('Face B present')
sheet2 = wb2.add_sheet('Face B missing')

#putting headers of subID in each excel sheet
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

#SELECTING QUALTRICS FILE
#listing files in measure_dir and selecting top option (there should only be 1 file in each dir -- if not, there could be issues)
dir_file = [f for f in listdir(measure_dir) if isfile(join(measure_dir, f))]
dir_file = dir_file[0]
#extracting the top file from measure_dir and putting into zip_temp_dir
with ZipFile(measure_dir + dir_file, 'r') as zip_ref:
    zip_ref.extractall(unzip_temp_dir)
#listing files in unzip_temp_dir (there should only be 1 file in each dir -- if not, there could be issues)
filename = [f for f in listdir(unzip_temp_dir) if isfile(join(unzip_temp_dir, f))]
#selecting file
filename = filename[0]

#READING IN QUALTRICS FILE
#creating a list of 1 header name. Only this 1 column will be read in
col_list_qual = [id_column]
#reading in the qualtrics file with only the header specified
df_qual = pd.read_csv(r'' + unzip_temp_dir + filename,  usecols=col_list_qual)

#READING IN REDCAP FILE
#reading in the redcap file -- this is just for SIGT to check if SIGT was needed
col_list_redcap = ["record_id", session, "aol_phit_id", "aol_task_id"]
df_redcap = pd.read_csv(r'' + redcap_dir + redcap_file,  usecols=col_list_redcap)

#turning the imported excel columns into lists so we can manipulate the data
df_qual_id = df_qual[id_column].tolist()
df_red_id = df_redcap["record_id"].tolist()
df_red_ses_check = df_redcap[session].tolist()
df_phit_id = df_redcap["aol_phit_id"].tolist()
df_task_id = df_redcap["aol_task_id"].tolist()
            
for x in df_updating_subs:
    if (int(x[5]) % 2) == 0:  #checking if last digit of subID is even
        loc = df_red_id.index(x) #finding location of sub in redcap file
        if df_red_ses_check[loc]==2: 
            if x in df_qual_id: 
                place_counter_complete=place_counter_complete+1 
                sheet1.write(place_counter_complete, 0, x) 
            if x not in df_qual_id: 
                if df_phit_id[loc] in df_qual_id: #checking if PHIT ID is in qualtrics measure
                    place_counter_complete=place_counter_complete+1 
                    sheet1.write(place_counter_complete, 0, x) 
                if df_phit_id[loc] not in df_qual_id: 
                    if df_task_id[loc] in df_qual_id: #checking if TASK ID is in qualtrics measure
                        place_counter_missing=place_counter_missing+1
                        sheet2.write(place_counter_missing, 0, x)
        
#saving the workborks .xlsx files        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

os.remove(unzip_temp_dir + filename)

In [78]:
#8. Lie video task A

#setting dir for this measure and identifying subID column in this qualtrics file
measure_dir = video_a_dir
id_column = "ID"
session = follow_up_1

#creating the sheets in the excel file where output will be written
sheet1 = wb1.add_sheet('Video A present')
sheet2 = wb2.add_sheet('Video A missing')

#putting headers of subID in each excel sheet
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

#SELECTING QUALTRICS FILE
#listing files in measure_dir and selecting top option (there should only be 1 file in each dir -- if not, there could be issues)
dir_file = [f for f in listdir(measure_dir) if isfile(join(measure_dir, f))]
dir_file = dir_file[0]
#extracting the top file from measure_dir and putting into zip_temp_dir
with ZipFile(measure_dir + dir_file, 'r') as zip_ref:
    zip_ref.extractall(unzip_temp_dir)
#listing files in unzip_temp_dir (there should only be 1 file in each dir -- if not, there could be issues)
filename = [f for f in listdir(unzip_temp_dir) if isfile(join(unzip_temp_dir, f))]
#selecting file
filename = filename[0]

#READING IN QUALTRICS FILE
#creating a list of 1 header name. Only this 1 column will be read in
col_list_qual = [id_column]
#reading in the qualtrics file with only the header specified
df_qual = pd.read_csv(r'' + unzip_temp_dir + filename,  usecols=col_list_qual)

#READING IN REDCAP FILE
#reading in the redcap file -- this is just for SIGT to check if SIGT was needed
col_list_redcap = ["record_id", session, "aol_phit_id", "aol_task_id"]
df_redcap = pd.read_csv(r'' + redcap_dir + redcap_file,  usecols=col_list_redcap)

#turning the imported excel columns into lists so we can manipulate the data
df_qual_id = df_qual[id_column].tolist()
df_red_id = df_redcap["record_id"].tolist()
df_red_ses_check = df_redcap[session].tolist()
df_phit_id = df_redcap["aol_phit_id"].tolist()
df_task_id = df_redcap["aol_task_id"].tolist()
            
for x in df_updating_subs:
    if (int(x[5]) % 2) != 0:  #checking if last digit of subID is odd
        loc = df_red_id.index(x) #finding location of sub in redcap file
        if df_red_ses_check[loc]==2: 
            if x in df_qual_id: 
                place_counter_complete=place_counter_complete+1 
                sheet1.write(place_counter_complete, 0, x) 
            if x not in df_qual_id: 
                if df_phit_id[loc] in df_qual_id: #checking if PHIT ID is in qualtrics measure
                    place_counter_complete=place_counter_complete+1 
                    sheet1.write(place_counter_complete, 0, x) 
                if df_phit_id[loc] not in df_qual_id: 
                    if df_task_id[loc] in df_qual_id: #checking if TASK ID is in qualtrics measure
                        place_counter_missing=place_counter_missing+1
                        sheet2.write(place_counter_missing, 0, x)
        
#saving the workborks .xlsx files        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

os.remove(unzip_temp_dir + filename)

In [79]:
#9. Lie video task B

#setting dir for this measure and identifying subID column in this qualtrics file
measure_dir = video_b_dir
id_column = "ID"
session = follow_up_1

#creating the sheets in the excel file where output will be written
sheet1 = wb1.add_sheet('Video B present')
sheet2 = wb2.add_sheet('Video B missing')

#putting headers of subID in each excel sheet
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

#SELECTING QUALTRICS FILE
#listing files in measure_dir and selecting top option (there should only be 1 file in each dir -- if not, there could be issues)
dir_file = [f for f in listdir(measure_dir) if isfile(join(measure_dir, f))]
dir_file = dir_file[0]
#extracting the top file from measure_dir and putting into zip_temp_dir
with ZipFile(measure_dir + dir_file, 'r') as zip_ref:
    zip_ref.extractall(unzip_temp_dir)
#listing files in unzip_temp_dir (there should only be 1 file in each dir -- if not, there could be issues)
filename = [f for f in listdir(unzip_temp_dir) if isfile(join(unzip_temp_dir, f))]
#selecting file
filename = filename[0]

#READING IN QUALTRICS FILE
#creating a list of 1 header name. Only this 1 column will be read in
col_list_qual = [id_column]
#reading in the qualtrics file with only the header specified
df_qual = pd.read_csv(r'' + unzip_temp_dir + filename,  usecols=col_list_qual)

#READING IN REDCAP FILE
#reading in the redcap file -- this is just for SIGT to check if SIGT was needed
col_list_redcap = ["record_id", session, "aol_phit_id", "aol_task_id"]
df_redcap = pd.read_csv(r'' + redcap_dir + redcap_file,  usecols=col_list_redcap)

#turning the imported excel columns into lists so we can manipulate the data
df_qual_id = df_qual[id_column].tolist()
df_red_id = df_redcap["record_id"].tolist()
df_red_ses_check = df_redcap[session].tolist()
df_phit_id = df_redcap["aol_phit_id"].tolist()
df_task_id = df_redcap["aol_task_id"].tolist()
            
for x in df_updating_subs:
    if (int(x[5]) % 2) != 0:  #checking if last digit of subID is odd
        loc = df_red_id.index(x) #finding location of sub in redcap file
        if df_red_ses_check[loc]==2: 
            if x in df_qual_id: 
                place_counter_complete=place_counter_complete+1 
                sheet1.write(place_counter_complete, 0, x) 
            if x not in df_qual_id: 
                if df_phit_id[loc] in df_qual_id: #checking if PHIT ID is in qualtrics measure
                    place_counter_complete=place_counter_complete+1 
                    sheet1.write(place_counter_complete, 0, x) 
                if df_phit_id[loc] not in df_qual_id: 
                    if df_task_id[loc] in df_qual_id: #checking if TASK ID is in qualtrics measure
                        place_counter_missing=place_counter_missing+1
                        sheet2.write(place_counter_missing, 0, x)
        
#saving the workborks .xlsx files        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

os.remove(unzip_temp_dir + filename)

In [80]:
#10. Picture task

#setting dir for this measure and identifying subID column in this qualtrics file
measure_dir = picture_dir
id_column = "Q36"
session = follow_up_1

#creating the sheets in the excel file where output will be written
sheet1 = wb1.add_sheet('Picture task present')
sheet2 = wb2.add_sheet('Picture task missing')

#putting headers of subID in each excel sheet
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

#SELECTING QUALTRICS FILE
#listing files in measure_dir and selecting top option (there should only be 1 file in each dir -- if not, there could be issues)
dir_file = [f for f in listdir(measure_dir) if isfile(join(measure_dir, f))]
dir_file = dir_file[0]
#extracting the top file from measure_dir and putting into zip_temp_dir
with ZipFile(measure_dir + dir_file, 'r') as zip_ref:
    zip_ref.extractall(unzip_temp_dir)
#listing files in unzip_temp_dir (there should only be 1 file in each dir -- if not, there could be issues)
filename = [f for f in listdir(unzip_temp_dir) if isfile(join(unzip_temp_dir, f))]
#selecting file
filename = filename[0]

#READING IN QUALTRICS FILE
#creating a list of 1 header name. Only this 1 column will be read in
col_list_qual = [id_column]
#reading in the qualtrics file with only the header specified
df_qual = pd.read_csv(r'' + unzip_temp_dir + filename,  usecols=col_list_qual)

#READING IN REDCAP FILE
#reading in the redcap file -- this is just for SIGT to check if SIGT was needed
col_list_redcap = ["record_id", session, "aol_phit_id", "aol_task_id"]
df_redcap = pd.read_csv(r'' + redcap_dir + redcap_file,  usecols=col_list_redcap)

#turning the imported excel columns into lists so we can manipulate the data
df_qual_id = df_qual[id_column].tolist()
df_red_id = df_redcap["record_id"].tolist()
df_red_ses_check = df_redcap[session].tolist()
df_phit_id = df_redcap["aol_phit_id"].tolist()
df_task_id = df_redcap["aol_task_id"].tolist()


for x in df_updating_subs:
    loc = df_red_id.index(x) #finding location of sub in redcap file
    if df_red_ses_check[loc]==2: 
        if x in df_qual_id: 
            place_counter_complete=place_counter_complete+1 
            sheet1.write(place_counter_complete, 0, x) 
        if x not in df_qual_id: 
            if df_phit_id[loc] in df_qual_id: #checking if PHIT ID is in qualtrics measure
                place_counter_complete=place_counter_complete+1 
                sheet1.write(place_counter_complete, 0, x) 
            if df_phit_id[loc] not in df_qual_id: 
                if df_task_id[loc] in df_qual_id: #checking if TASK ID is in qualtrics measure
                    place_counter_complete=place_counter_complete+1 
                    sheet1.write(place_counter_complete, 0, x) 
                if df_task_id[loc] not in df_qual_id: 
                    place_counter_missing=place_counter_missing+1
                    sheet2.write(place_counter_missing, 0, x)
        
#saving the workborks .xlsx files        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

os.remove(unzip_temp_dir + filename)

In [81]:
#11. Shipley vocab

#setting dir for this measure and identifying subID column in this qualtrics file
measure_dir = vocab_dir
id_column = "ID"
session = follow_up_1

#creating the sheets in the excel file where output will be written
sheet1 = wb1.add_sheet('Vocab task present')
sheet2 = wb2.add_sheet('Vocab task missing')

#putting headers of subID in each excel sheet
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

#SELECTING QUALTRICS FILE
#listing files in measure_dir and selecting top option (there should only be 1 file in each dir -- if not, there could be issues)
dir_file = [f for f in listdir(measure_dir) if isfile(join(measure_dir, f))]
dir_file = dir_file[0]
#extracting the top file from measure_dir and putting into zip_temp_dir
with ZipFile(measure_dir + dir_file, 'r') as zip_ref:
    zip_ref.extractall(unzip_temp_dir)
#listing files in unzip_temp_dir (there should only be 1 file in each dir -- if not, there could be issues)
filename = [f for f in listdir(unzip_temp_dir) if isfile(join(unzip_temp_dir, f))]
#selecting file
filename = filename[0]

#READING IN QUALTRICS FILE
#creating a list of 1 header name. Only this 1 column will be read in
col_list_qual = [id_column]
#reading in the qualtrics file with only the header specified
df_qual = pd.read_csv(r'' + unzip_temp_dir + filename,  usecols=col_list_qual)

#READING IN REDCAP FILE
#reading in the redcap file -- this is just for SIGT to check if SIGT was needed
col_list_redcap = ["record_id", session, "aol_phit_id", "aol_task_id"]
df_redcap = pd.read_csv(r'' + redcap_dir + redcap_file,  usecols=col_list_redcap)

#turning the imported excel columns into lists so we can manipulate the data
df_qual_id = df_qual[id_column].tolist()
df_red_id = df_redcap["record_id"].tolist()
df_red_ses_check = df_redcap[session].tolist()
df_phit_id = df_redcap["aol_phit_id"].tolist()
df_task_id = df_redcap["aol_task_id"].tolist()


for x in df_updating_subs:
    loc = df_red_id.index(x) #finding location of sub in redcap file
    if df_red_ses_check[loc]==2: 
        if x in df_qual_id: 
            place_counter_complete=place_counter_complete+1 
            sheet1.write(place_counter_complete, 0, x) 
        if x not in df_qual_id: 
            if df_phit_id[loc] in df_qual_id: #checking if PHIT ID is in qualtrics measure
                place_counter_complete=place_counter_complete+1 
                sheet1.write(place_counter_complete, 0, x) 
            if df_phit_id[loc] not in df_qual_id: 
                if df_task_id[loc] in df_qual_id: #checking if TASK ID is in qualtrics measure
                    place_counter_complete=place_counter_complete+1 
                    sheet1.write(place_counter_complete, 0, x) 
                if df_task_id[loc] not in df_qual_id: 
                    place_counter_missing=place_counter_missing+1
                    sheet2.write(place_counter_missing, 0, x)
        
#saving the workborks .xlsx files        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

os.remove(unzip_temp_dir + filename)

In [82]:
#12. FNT

#setting dir for this measure and identifying subID column in this qualtrics file
measure_dir = fnt_dir
id_column = "Q136"
session = follow_up_1

#creating the sheets in the excel file where output will be written
sheet1 = wb1.add_sheet('FNT present')
sheet2 = wb2.add_sheet('FNT missing')

#putting headers of subID in each excel sheet
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

#SELECTING QUALTRICS FILE
#listing files in measure_dir and selecting top option (there should only be 1 file in each dir -- if not, there could be issues)
dir_file = [f for f in listdir(measure_dir) if isfile(join(measure_dir, f))]
dir_file = dir_file[0]
#extracting the top file from measure_dir and putting into zip_temp_dir
with ZipFile(measure_dir + dir_file, 'r') as zip_ref:
    zip_ref.extractall(unzip_temp_dir)
#listing files in unzip_temp_dir (there should only be 1 file in each dir -- if not, there could be issues)
filename = [f for f in listdir(unzip_temp_dir) if isfile(join(unzip_temp_dir, f))]
#selecting file
filename = filename[0]

#READING IN QUALTRICS FILE
#creating a list of 1 header name. Only this 1 column will be read in
col_list_qual = [id_column]
#reading in the qualtrics file with only the header specified
df_qual = pd.read_csv(r'' + unzip_temp_dir + filename,  usecols=col_list_qual)

#READING IN REDCAP FILE
#reading in the redcap file -- this is just for SIGT to check if SIGT was needed
col_list_redcap = ["record_id", session, "aol_phit_id", "aol_task_id"]
df_redcap = pd.read_csv(r'' + redcap_dir + redcap_file,  usecols=col_list_redcap)

#turning the imported excel columns into lists so we can manipulate the data
df_qual_id = df_qual[id_column].tolist()
df_red_id = df_redcap["record_id"].tolist()
df_red_ses_check = df_redcap[session].tolist()
df_phit_id = df_redcap["aol_phit_id"].tolist()
df_task_id = df_redcap["aol_task_id"].tolist()


for x in df_updating_subs:
    loc = df_red_id.index(x) #finding location of sub in redcap file
    if df_red_ses_check[loc]==2: 
        if x in df_qual_id: 
            place_counter_complete=place_counter_complete+1 
            sheet1.write(place_counter_complete, 0, x) 
        if x not in df_qual_id: 
            if df_phit_id[loc] in df_qual_id: #checking if PHIT ID is in qualtrics measure
                place_counter_complete=place_counter_complete+1 
                sheet1.write(place_counter_complete, 0, x) 
            if df_phit_id[loc] not in df_qual_id: 
                if df_task_id[loc] in df_qual_id: #checking if TASK ID is in qualtrics measure
                    place_counter_complete=place_counter_complete+1 
                    sheet1.write(place_counter_complete, 0, x) 
                if df_task_id[loc] not in df_qual_id: 
                    place_counter_missing=place_counter_missing+1
                    sheet2.write(place_counter_missing, 0, x)
        
#saving the workborks .xlsx files        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

os.remove(unzip_temp_dir + filename)

In [83]:
#13. Temporal discounting gains

#setting dir for this measure and identifying subID column in this qualtrics file
measure_dir = temporal_gains_dir
id_column = "ID"
session = follow_up_1

#creating the sheets in the excel file where output will be written
sheet1 = wb1.add_sheet('Temp disc gains present')
sheet2 = wb2.add_sheet('Temp disc gains missing')

#putting headers of subID in each excel sheet
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

#SELECTING QUALTRICS FILE
#listing files in measure_dir and selecting top option (there should only be 1 file in each dir -- if not, there could be issues)
dir_file = [f for f in listdir(measure_dir) if isfile(join(measure_dir, f))]
dir_file = dir_file[0]
#extracting the top file from measure_dir and putting into zip_temp_dir
with ZipFile(measure_dir + dir_file, 'r') as zip_ref:
    zip_ref.extractall(unzip_temp_dir)
#listing files in unzip_temp_dir (there should only be 1 file in each dir -- if not, there could be issues)
filename = [f for f in listdir(unzip_temp_dir) if isfile(join(unzip_temp_dir, f))]
#selecting file
filename = filename[0]

#READING IN QUALTRICS FILE
#creating a list of 1 header name. Only this 1 column will be read in
col_list_qual = [id_column]
#reading in the qualtrics file with only the header specified
df_qual = pd.read_csv(r'' + unzip_temp_dir + filename,  usecols=col_list_qual)

#READING IN REDCAP FILE
#reading in the redcap file -- this is just for SIGT to check if SIGT was needed
col_list_redcap = ["record_id", session, "aol_phit_id", "aol_task_id"]
df_redcap = pd.read_csv(r'' + redcap_dir + redcap_file,  usecols=col_list_redcap)

#turning the imported excel columns into lists so we can manipulate the data
df_qual_id = df_qual[id_column].tolist()
df_red_id = df_redcap["record_id"].tolist()
df_red_ses_check = df_redcap[session].tolist()
df_phit_id = df_redcap["aol_phit_id"].tolist()
df_task_id = df_redcap["aol_task_id"].tolist()
            
for x in df_updating_subs:
    if (int(x[5]) % 2) != 0:  #checking if last digit of subID is odd
        loc = df_red_id.index(x) #finding location of sub in redcap file
        if df_red_ses_check[loc]==2: 
            if x in df_qual_id: 
                place_counter_complete=place_counter_complete+1 
                sheet1.write(place_counter_complete, 0, x) 
            if x not in df_qual_id: 
                if df_phit_id[loc] in df_qual_id: #checking if PHIT ID is in qualtrics measure
                    place_counter_complete=place_counter_complete+1 
                    sheet1.write(place_counter_complete, 0, x) 
                if df_phit_id[loc] not in df_qual_id: 
                    if df_task_id[loc] in df_qual_id: #checking if TASK ID is in qualtrics measure
                        place_counter_missing=place_counter_missing+1
                        sheet2.write(place_counter_missing, 0, x)
        
#saving the workborks .xlsx files        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

os.remove(unzip_temp_dir + filename)

In [84]:
#14. Temporal discounting losses

#setting dir for this measure and identifying subID column in this qualtrics file
measure_dir = temporal_losses_dir
id_column = "ID"
session = follow_up_1

#creating the sheets in the excel file where output will be written
sheet1 = wb1.add_sheet('Temp disc losses present')
sheet2 = wb2.add_sheet('Temp disc losses missing')

#putting headers of subID in each excel sheet
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

#SELECTING QUALTRICS FILE
#listing files in measure_dir and selecting top option (there should only be 1 file in each dir -- if not, there could be issues)
dir_file = [f for f in listdir(measure_dir) if isfile(join(measure_dir, f))]
dir_file = dir_file[0]
#extracting the top file from measure_dir and putting into zip_temp_dir
with ZipFile(measure_dir + dir_file, 'r') as zip_ref:
    zip_ref.extractall(unzip_temp_dir)
#listing files in unzip_temp_dir (there should only be 1 file in each dir -- if not, there could be issues)
filename = [f for f in listdir(unzip_temp_dir) if isfile(join(unzip_temp_dir, f))]
#selecting file
filename = filename[0]

#READING IN QUALTRICS FILE
#creating a list of 1 header name. Only this 1 column will be read in
col_list_qual = [id_column]
#reading in the qualtrics file with only the header specified
df_qual = pd.read_csv(r'' + unzip_temp_dir + filename,  usecols=col_list_qual)

#READING IN REDCAP FILE
#reading in the redcap file -- this is just for SIGT to check if SIGT was needed
col_list_redcap = ["record_id", session, "aol_phit_id", "aol_task_id"]
df_redcap = pd.read_csv(r'' + redcap_dir + redcap_file,  usecols=col_list_redcap)

#turning the imported excel columns into lists so we can manipulate the data
df_qual_id = df_qual[id_column].tolist()
df_red_id = df_redcap["record_id"].tolist()
df_red_ses_check = df_redcap[session].tolist()
df_phit_id = df_redcap["aol_phit_id"].tolist()
df_task_id = df_redcap["aol_task_id"].tolist()
            
for x in df_updating_subs:
    if (int(x[5]) % 2) == 0:  #checking if last digit of subID is odd
        loc = df_red_id.index(x) #finding location of sub in redcap file
        if df_red_ses_check[loc]==2: 
            if x in df_qual_id: 
                place_counter_complete=place_counter_complete+1 
                sheet1.write(place_counter_complete, 0, x) 
            if x not in df_qual_id: 
                if df_phit_id[loc] in df_qual_id: #checking if PHIT ID is in qualtrics measure
                    place_counter_complete=place_counter_complete+1 
                    sheet1.write(place_counter_complete, 0, x) 
                if df_phit_id[loc] not in df_qual_id: 
                    if df_task_id[loc] in df_qual_id: #checking if TASK ID is in qualtrics measure
                        place_counter_missing=place_counter_missing+1
                        sheet2.write(place_counter_missing, 0, x)
        
#saving the workborks .xlsx files        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

os.remove(unzip_temp_dir + filename)

In [85]:
#15. Assessment 1

#setting dir for this measure and identifying subID column in this qualtrics file
measure_dir = assessment_1_dir
id_column = "ID"
session = follow_up_2

#creating the sheets in the excel file where output will be written
sheet1 = wb1.add_sheet('Assessment 1 present')
sheet2 = wb2.add_sheet('Assessment 1 missing')

#putting headers of subID in each excel sheet
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

#SELECTING QUALTRICS FILE
#listing files in measure_dir and selecting top option (there should only be 1 file in each dir -- if not, there could be issues)
dir_file = [f for f in listdir(measure_dir) if isfile(join(measure_dir, f))]
dir_file = dir_file[0]
#extracting the top file from measure_dir and putting into zip_temp_dir
with ZipFile(measure_dir + dir_file, 'r') as zip_ref:
    zip_ref.extractall(unzip_temp_dir)
#listing files in unzip_temp_dir (there should only be 1 file in each dir -- if not, there could be issues)
filename = [f for f in listdir(unzip_temp_dir) if isfile(join(unzip_temp_dir, f))]
#selecting file
filename = filename[0]

#READING IN QUALTRICS FILE
#creating a list of 1 header name. Only this 1 column will be read in
col_list_qual = [id_column]
#reading in the qualtrics file with only the header specified
df_qual = pd.read_csv(r'' + unzip_temp_dir + filename,  usecols=col_list_qual)

#READING IN REDCAP FILE
#reading in the redcap file -- this is just for SIGT to check if SIGT was needed
col_list_redcap = ["record_id", session, "aol_phit_id", "aol_task_id"]
df_redcap = pd.read_csv(r'' + redcap_dir + redcap_file,  usecols=col_list_redcap)

#turning the imported excel columns into lists so we can manipulate the data
df_qual_id = df_qual[id_column].tolist()
df_red_id = df_redcap["record_id"].tolist()
df_red_ses_check = df_redcap[session].tolist()
df_phit_id = df_redcap["aol_phit_id"].tolist()
df_task_id = df_redcap["aol_task_id"].tolist()
            
for x in df_updating_subs:
    loc = df_red_id.index(x) #finding location of sub in redcap file
    if df_red_ses_check[loc]==2: 
        if x in df_qual_id: 
            place_counter_complete=place_counter_complete+1 
            sheet1.write(place_counter_complete, 0, x) 
        if x not in df_qual_id: 
            if df_phit_id[loc] in df_qual_id: #checking if PHIT ID is in qualtrics measure
                place_counter_complete=place_counter_complete+1 
                sheet1.write(place_counter_complete, 0, x) 
            if df_phit_id[loc] not in df_qual_id: 
                if df_task_id[loc] in df_qual_id: #checking if TASK ID is in qualtrics measure
                    place_counter_missing=place_counter_missing+1
                    sheet2.write(place_counter_missing, 0, x)
        
#saving the workborks .xlsx files        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

os.remove(unzip_temp_dir + filename)

In [86]:
#16. Assessment 2

#setting dir for this measure and identifying subID column in this qualtrics file
measure_dir = assessment_2_dir
id_column = "ID"
session = follow_up_2

#creating the sheets in the excel file where output will be written
sheet1 = wb1.add_sheet('Assessment 2 present')
sheet2 = wb2.add_sheet('Assessment 2 missing')

#putting headers of subID in each excel sheet
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

#SELECTING QUALTRICS FILE
#listing files in measure_dir and selecting top option (there should only be 1 file in each dir -- if not, there could be issues)
dir_file = [f for f in listdir(measure_dir) if isfile(join(measure_dir, f))]
dir_file = dir_file[0]
#extracting the top file from measure_dir and putting into zip_temp_dir
with ZipFile(measure_dir + dir_file, 'r') as zip_ref:
    zip_ref.extractall(unzip_temp_dir)
#listing files in unzip_temp_dir (there should only be 1 file in each dir -- if not, there could be issues)
filename = [f for f in listdir(unzip_temp_dir) if isfile(join(unzip_temp_dir, f))]
#selecting file
filename = filename[0]

#READING IN QUALTRICS FILE
#creating a list of 1 header name. Only this 1 column will be read in
col_list_qual = [id_column]
#reading in the qualtrics file with only the header specified
df_qual = pd.read_csv(r'' + unzip_temp_dir + filename,  usecols=col_list_qual)

#READING IN REDCAP FILE
#reading in the redcap file -- this is just for SIGT to check if SIGT was needed
col_list_redcap = ["record_id", session, "aol_phit_id", "aol_task_id"]
df_redcap = pd.read_csv(r'' + redcap_dir + redcap_file,  usecols=col_list_redcap)

#turning the imported excel columns into lists so we can manipulate the data
df_qual_id = df_qual[id_column].tolist()
df_red_id = df_redcap["record_id"].tolist()
df_red_ses_check = df_redcap[session].tolist()
df_phit_id = df_redcap["aol_phit_id"].tolist()
df_task_id = df_redcap["aol_task_id"].tolist()
            
for x in df_updating_subs:
    loc = df_red_id.index(x) #finding location of sub in redcap file
    if df_red_ses_check[loc]==2: 
        if x in df_qual_id: 
            place_counter_complete=place_counter_complete+1 
            sheet1.write(place_counter_complete, 0, x) 
        if x not in df_qual_id: 
            if df_phit_id[loc] in df_qual_id: #checking if PHIT ID is in qualtrics measure
                place_counter_complete=place_counter_complete+1 
                sheet1.write(place_counter_complete, 0, x) 
            if df_phit_id[loc] not in df_qual_id: 
                if df_task_id[loc] in df_qual_id: #checking if TASK ID is in qualtrics measure
                    place_counter_missing=place_counter_missing+1
                    sheet2.write(place_counter_missing, 0, x)
        
#saving the workborks .xlsx files        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

os.remove(unzip_temp_dir + filename)

In [87]:
#17. Assessment 3

#setting dir for this measure and identifying subID column in this qualtrics file
measure_dir = assessment_3_dir
id_column = "ID"
session = follow_up_2

#creating the sheets in the excel file where output will be written
sheet1 = wb1.add_sheet('Assessment 3 present')
sheet2 = wb2.add_sheet('Assessment 3 missing')

#putting headers of subID in each excel sheet
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

#SELECTING QUALTRICS FILE
#listing files in measure_dir and selecting top option (there should only be 1 file in each dir -- if not, there could be issues)
dir_file = [f for f in listdir(measure_dir) if isfile(join(measure_dir, f))]
dir_file = dir_file[0]
#extracting the top file from measure_dir and putting into zip_temp_dir
with ZipFile(measure_dir + dir_file, 'r') as zip_ref:
    zip_ref.extractall(unzip_temp_dir)
#listing files in unzip_temp_dir (there should only be 1 file in each dir -- if not, there could be issues)
filename = [f for f in listdir(unzip_temp_dir) if isfile(join(unzip_temp_dir, f))]
#selecting file
filename = filename[0]

#READING IN QUALTRICS FILE
#creating a list of 1 header name. Only this 1 column will be read in
col_list_qual = [id_column]
#reading in the qualtrics file with only the header specified
df_qual = pd.read_csv(r'' + unzip_temp_dir + filename,  usecols=col_list_qual)

#READING IN REDCAP FILE
#reading in the redcap file -- this is just for SIGT to check if SIGT was needed
col_list_redcap = ["record_id", session, "aol_phit_id", "aol_task_id"]
df_redcap = pd.read_csv(r'' + redcap_dir + redcap_file,  usecols=col_list_redcap)

#turning the imported excel columns into lists so we can manipulate the data
df_qual_id = df_qual[id_column].tolist()
df_red_id = df_redcap["record_id"].tolist()
df_red_ses_check = df_redcap[session].tolist()
df_phit_id = df_redcap["aol_phit_id"].tolist()
df_task_id = df_redcap["aol_task_id"].tolist()
            
for x in df_updating_subs:
    loc = df_red_id.index(x) #finding location of sub in redcap file
    if df_red_ses_check[loc]==2: 
        if x in df_qual_id: 
            place_counter_complete=place_counter_complete+1 
            sheet1.write(place_counter_complete, 0, x) 
        if x not in df_qual_id: 
            if df_phit_id[loc] in df_qual_id: #checking if PHIT ID is in qualtrics measure
                place_counter_complete=place_counter_complete+1 
                sheet1.write(place_counter_complete, 0, x) 
            if df_phit_id[loc] not in df_qual_id: 
                if df_task_id[loc] in df_qual_id: #checking if TASK ID is in qualtrics measure
                    place_counter_missing=place_counter_missing+1
                    sheet2.write(place_counter_missing, 0, x)
        
#saving the workborks .xlsx files        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

os.remove(unzip_temp_dir + filename)

In [88]:
#18. Assessment 4

#setting dir for this measure and identifying subID column in this qualtrics file
measure_dir = assessment_4_dir
id_column = "ID"
session = follow_up_2

#creating the sheets in the excel file where output will be written
sheet1 = wb1.add_sheet('Assessment 4 present')
sheet2 = wb2.add_sheet('Assessment 4 missing')

#putting headers of subID in each excel sheet
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

#SELECTING QUALTRICS FILE
#listing files in measure_dir and selecting top option (there should only be 1 file in each dir -- if not, there could be issues)
dir_file = [f for f in listdir(measure_dir) if isfile(join(measure_dir, f))]
dir_file = dir_file[0]
#extracting the top file from measure_dir and putting into zip_temp_dir
with ZipFile(measure_dir + dir_file, 'r') as zip_ref:
    zip_ref.extractall(unzip_temp_dir)
#listing files in unzip_temp_dir (there should only be 1 file in each dir -- if not, there could be issues)
filename = [f for f in listdir(unzip_temp_dir) if isfile(join(unzip_temp_dir, f))]
#selecting file
filename = filename[0]

#READING IN QUALTRICS FILE
#creating a list of 1 header name. Only this 1 column will be read in
col_list_qual = [id_column]
#reading in the qualtrics file with only the header specified
df_qual = pd.read_csv(r'' + unzip_temp_dir + filename,  usecols=col_list_qual)

#READING IN REDCAP FILE
#reading in the redcap file -- this is just for SIGT to check if SIGT was needed
col_list_redcap = ["record_id", session, "aol_phit_id", "aol_task_id"]
df_redcap = pd.read_csv(r'' + redcap_dir + redcap_file,  usecols=col_list_redcap)

#turning the imported excel columns into lists so we can manipulate the data
df_qual_id = df_qual[id_column].tolist()
df_red_id = df_redcap["record_id"].tolist()
df_red_ses_check = df_redcap[session].tolist()
df_phit_id = df_redcap["aol_phit_id"].tolist()
df_task_id = df_redcap["aol_task_id"].tolist()
            
for x in df_updating_subs:
    loc = df_red_id.index(x) #finding location of sub in redcap file
    if df_red_ses_check[loc]==2: 
        if x in df_qual_id: 
            place_counter_complete=place_counter_complete+1 
            sheet1.write(place_counter_complete, 0, x) 
        if x not in df_qual_id: 
            if df_phit_id[loc] in df_qual_id: #checking if PHIT ID is in qualtrics measure
                place_counter_complete=place_counter_complete+1 
                sheet1.write(place_counter_complete, 0, x) 
            if df_phit_id[loc] not in df_qual_id: 
                if df_task_id[loc] in df_qual_id: #checking if TASK ID is in qualtrics measure
                    place_counter_missing=place_counter_missing+1
                    sheet2.write(place_counter_missing, 0, x)
        
#saving the workborks .xlsx files        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

os.remove(unzip_temp_dir + filename)

In [89]:
#19. Assessment 5

#setting dir for this measure and identifying subID column in this qualtrics file
measure_dir = assessment_5_dir
id_column = "ID"
session = follow_up_2

#creating the sheets in the excel file where output will be written
sheet1 = wb1.add_sheet('Assessment 5 present')
sheet2 = wb2.add_sheet('Assessment 5 missing')

#putting headers of subID in each excel sheet
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

#SELECTING QUALTRICS FILE
#listing files in measure_dir and selecting top option (there should only be 1 file in each dir -- if not, there could be issues)
dir_file = [f for f in listdir(measure_dir) if isfile(join(measure_dir, f))]
dir_file = dir_file[0]
#extracting the top file from measure_dir and putting into zip_temp_dir
with ZipFile(measure_dir + dir_file, 'r') as zip_ref:
    zip_ref.extractall(unzip_temp_dir)
#listing files in unzip_temp_dir (there should only be 1 file in each dir -- if not, there could be issues)
filename = [f for f in listdir(unzip_temp_dir) if isfile(join(unzip_temp_dir, f))]
#selecting file
filename = filename[0]

#READING IN QUALTRICS FILE
#creating a list of 1 header name. Only this 1 column will be read in
col_list_qual = [id_column]
#reading in the qualtrics file with only the header specified
df_qual = pd.read_csv(r'' + unzip_temp_dir + filename,  usecols=col_list_qual)

#READING IN REDCAP FILE
#reading in the redcap file -- this is just for SIGT to check if SIGT was needed
col_list_redcap = ["record_id", session, "aol_phit_id", "aol_task_id"]
df_redcap = pd.read_csv(r'' + redcap_dir + redcap_file,  usecols=col_list_redcap)

#turning the imported excel columns into lists so we can manipulate the data
df_qual_id = df_qual[id_column].tolist()
df_red_id = df_redcap["record_id"].tolist()
df_red_ses_check = df_redcap[session].tolist()
df_phit_id = df_redcap["aol_phit_id"].tolist()
df_task_id = df_redcap["aol_task_id"].tolist()
            
for x in df_updating_subs:
    loc = df_red_id.index(x) #finding location of sub in redcap file
    if df_red_ses_check[loc]==2: 
        if x in df_qual_id: 
            place_counter_complete=place_counter_complete+1 
            sheet1.write(place_counter_complete, 0, x) 
        if x not in df_qual_id: 
            if df_phit_id[loc] in df_qual_id: #checking if PHIT ID is in qualtrics measure
                place_counter_complete=place_counter_complete+1 
                sheet1.write(place_counter_complete, 0, x) 
            if df_phit_id[loc] not in df_qual_id: 
                if df_task_id[loc] in df_qual_id: #checking if TASK ID is in qualtrics measure
                    place_counter_missing=place_counter_missing+1
                    sheet2.write(place_counter_missing, 0, x)
        
#saving the workborks .xlsx files        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

os.remove(unzip_temp_dir + filename)

In [90]:
#20. Assessment 6

#setting dir for this measure and identifying subID column in this qualtrics file
measure_dir = assessment_6_dir
id_column = "ID"
session = follow_up_2

#creating the sheets in the excel file where output will be written
sheet1 = wb1.add_sheet('Assessment 6 present')
sheet2 = wb2.add_sheet('Assessment 6 missing')

#putting headers of subID in each excel sheet
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

#SELECTING QUALTRICS FILE
#listing files in measure_dir and selecting top option (there should only be 1 file in each dir -- if not, there could be issues)
dir_file = [f for f in listdir(measure_dir) if isfile(join(measure_dir, f))]
dir_file = dir_file[0]
#extracting the top file from measure_dir and putting into zip_temp_dir
with ZipFile(measure_dir + dir_file, 'r') as zip_ref:
    zip_ref.extractall(unzip_temp_dir)
#listing files in unzip_temp_dir (there should only be 1 file in each dir -- if not, there could be issues)
filename = [f for f in listdir(unzip_temp_dir) if isfile(join(unzip_temp_dir, f))]
#selecting file
filename = filename[0]

#READING IN QUALTRICS FILE
#creating a list of 1 header name. Only this 1 column will be read in
col_list_qual = [id_column]
#reading in the qualtrics file with only the header specified
df_qual = pd.read_csv(r'' + unzip_temp_dir + filename,  usecols=col_list_qual)

#READING IN REDCAP FILE
#reading in the redcap file -- this is just for SIGT to check if SIGT was needed
col_list_redcap = ["record_id", session, "aol_phit_id", "aol_task_id", "datacheck_si_additional"]
df_redcap = pd.read_csv(r'' + redcap_dir + redcap_file,  usecols=col_list_redcap)

#turning the imported excel columns into lists so we can manipulate the data
df_qual_id = df_qual[id_column].tolist()
df_red_id = df_redcap["record_id"].tolist()
df_red_ses_check = df_redcap[session].tolist()
df_phit_id = df_redcap["aol_phit_id"].tolist()
df_task_id = df_redcap["aol_task_id"].tolist()
df_additional_assessments = df_redcap["datacheck_si_additional"].tolist()
            
for x in df_updating_subs:
    loc = df_red_id.index(x) #finding location of sub in redcap file
    if df_additional_assessments[loc]==1:
        if df_red_ses_check[loc]==2: 
            if x in df_qual_id: 
                place_counter_complete=place_counter_complete+1 
                sheet1.write(place_counter_complete, 0, x) 
            if x not in df_qual_id: 
                if df_phit_id[loc] in df_qual_id: #checking if PHIT ID is in qualtrics measure
                    place_counter_complete=place_counter_complete+1 
                    sheet1.write(place_counter_complete, 0, x) 
                if df_phit_id[loc] not in df_qual_id: 
                    if df_task_id[loc] in df_qual_id: #checking if TASK ID is in qualtrics measure
                        place_counter_missing=place_counter_missing+1
                        sheet2.write(place_counter_missing, 0, x)
        
#saving the workborks .xlsx files        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

os.remove(unzip_temp_dir + filename)

In [91]:
#21. Assessment 7

#setting dir for this measure and identifying subID column in this qualtrics file
measure_dir = assessment_7_dir
id_column = "ID"
session = follow_up_2

#creating the sheets in the excel file where output will be written
sheet1 = wb1.add_sheet('Assessment 7 present')
sheet2 = wb2.add_sheet('Assessment 7 missing')

#putting headers of subID in each excel sheet
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

#SELECTING QUALTRICS FILE
#listing files in measure_dir and selecting top option (there should only be 1 file in each dir -- if not, there could be issues)
dir_file = [f for f in listdir(measure_dir) if isfile(join(measure_dir, f))]
dir_file = dir_file[0]
#extracting the top file from measure_dir and putting into zip_temp_dir
with ZipFile(measure_dir + dir_file, 'r') as zip_ref:
    zip_ref.extractall(unzip_temp_dir)
#listing files in unzip_temp_dir (there should only be 1 file in each dir -- if not, there could be issues)
filename = [f for f in listdir(unzip_temp_dir) if isfile(join(unzip_temp_dir, f))]
#selecting file
filename = filename[0]

#READING IN QUALTRICS FILE
#creating a list of 1 header name. Only this 1 column will be read in
col_list_qual = [id_column]
#reading in the qualtrics file with only the header specified
df_qual = pd.read_csv(r'' + unzip_temp_dir + filename,  usecols=col_list_qual)

#READING IN REDCAP FILE
#reading in the redcap file -- this is just for SIGT to check if SIGT was needed
col_list_redcap = ["record_id", session, "aol_phit_id", "aol_task_id", "datacheck_si_additional"]
df_redcap = pd.read_csv(r'' + redcap_dir + redcap_file,  usecols=col_list_redcap)

#turning the imported excel columns into lists so we can manipulate the data
df_qual_id = df_qual[id_column].tolist()
df_red_id = df_redcap["record_id"].tolist()
df_red_ses_check = df_redcap[session].tolist()
df_phit_id = df_redcap["aol_phit_id"].tolist()
df_task_id = df_redcap["aol_task_id"].tolist()
df_additional_assessments = df_redcap["datacheck_si_additional"].tolist()
            
for x in df_updating_subs:
    loc = df_red_id.index(x) #finding location of sub in redcap file
    if df_additional_assessments[loc]==1:
        if df_red_ses_check[loc]==2: 
            if x in df_qual_id: 
                place_counter_complete=place_counter_complete+1 
                sheet1.write(place_counter_complete, 0, x) 
            if x not in df_qual_id: 
                if df_phit_id[loc] in df_qual_id: #checking if PHIT ID is in qualtrics measure
                    place_counter_complete=place_counter_complete+1 
                    sheet1.write(place_counter_complete, 0, x) 
                if df_phit_id[loc] not in df_qual_id: 
                    if df_task_id[loc] in df_qual_id: #checking if TASK ID is in qualtrics measure
                        place_counter_missing=place_counter_missing+1
                        sheet2.write(place_counter_missing, 0, x)
        
#saving the workborks .xlsx files        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

os.remove(unzip_temp_dir + filename)

In [92]:
#22. Assessment 8

#setting dir for this measure and identifying subID column in this qualtrics file
measure_dir = assessment_8_dir
id_column = "ID"
session = follow_up_2

#creating the sheets in the excel file where output will be written
sheet1 = wb1.add_sheet('Assessment 8 present')
sheet2 = wb2.add_sheet('Assessment 8 missing')

#putting headers of subID in each excel sheet
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

#SELECTING QUALTRICS FILE
#listing files in measure_dir and selecting top option (there should only be 1 file in each dir -- if not, there could be issues)
dir_file = [f for f in listdir(measure_dir) if isfile(join(measure_dir, f))]
dir_file = dir_file[0]
#extracting the top file from measure_dir and putting into zip_temp_dir
with ZipFile(measure_dir + dir_file, 'r') as zip_ref:
    zip_ref.extractall(unzip_temp_dir)
#listing files in unzip_temp_dir (there should only be 1 file in each dir -- if not, there could be issues)
filename = [f for f in listdir(unzip_temp_dir) if isfile(join(unzip_temp_dir, f))]
#selecting file
filename = filename[0]

#READING IN QUALTRICS FILE
#creating a list of 1 header name. Only this 1 column will be read in
col_list_qual = [id_column]
#reading in the qualtrics file with only the header specified
df_qual = pd.read_csv(r'' + unzip_temp_dir + filename,  usecols=col_list_qual)

#READING IN REDCAP FILE
#reading in the redcap file -- this is just for SIGT to check if SIGT was needed
col_list_redcap = ["record_id", session, "aol_phit_id", "aol_task_id", "datacheck_si_additional"]
df_redcap = pd.read_csv(r'' + redcap_dir + redcap_file,  usecols=col_list_redcap)

#turning the imported excel columns into lists so we can manipulate the data
df_qual_id = df_qual[id_column].tolist()
df_red_id = df_redcap["record_id"].tolist()
df_red_ses_check = df_redcap[session].tolist()
df_phit_id = df_redcap["aol_phit_id"].tolist()
df_task_id = df_redcap["aol_task_id"].tolist()
df_additional_assessments = df_redcap["datacheck_si_additional"].tolist()
            
for x in df_updating_subs:
    loc = df_red_id.index(x) #finding location of sub in redcap file
    if df_additional_assessments[loc]==1:
        if df_red_ses_check[loc]==2: 
            if x in df_qual_id: 
                place_counter_complete=place_counter_complete+1 
                sheet1.write(place_counter_complete, 0, x) 
            if x not in df_qual_id: 
                if df_phit_id[loc] in df_qual_id: #checking if PHIT ID is in qualtrics measure
                    place_counter_complete=place_counter_complete+1 
                    sheet1.write(place_counter_complete, 0, x) 
                if df_phit_id[loc] not in df_qual_id: 
                    if df_task_id[loc] in df_qual_id: #checking if TASK ID is in qualtrics measure
                        place_counter_missing=place_counter_missing+1
                        sheet2.write(place_counter_missing, 0, x)
        
#saving the workborks .xlsx files        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

os.remove(unzip_temp_dir + filename)

In [93]:
#23. Assessment 9

#setting dir for this measure and identifying subID column in this qualtrics file
measure_dir = assessment_9_dir
id_column = "ID"
session = follow_up_2

#creating the sheets in the excel file where output will be written
sheet1 = wb1.add_sheet('Assessment 9 present')
sheet2 = wb2.add_sheet('Assessment 9 missing')

#putting headers of subID in each excel sheet
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

#SELECTING QUALTRICS FILE
#listing files in measure_dir and selecting top option (there should only be 1 file in each dir -- if not, there could be issues)
dir_file = [f for f in listdir(measure_dir) if isfile(join(measure_dir, f))]
dir_file = dir_file[0]
#extracting the top file from measure_dir and putting into zip_temp_dir
with ZipFile(measure_dir + dir_file, 'r') as zip_ref:
    zip_ref.extractall(unzip_temp_dir)
#listing files in unzip_temp_dir (there should only be 1 file in each dir -- if not, there could be issues)
filename = [f for f in listdir(unzip_temp_dir) if isfile(join(unzip_temp_dir, f))]
#selecting file
filename = filename[0]

#READING IN QUALTRICS FILE
#creating a list of 1 header name. Only this 1 column will be read in
col_list_qual = [id_column]
#reading in the qualtrics file with only the header specified
df_qual = pd.read_csv(r'' + unzip_temp_dir + filename,  usecols=col_list_qual)

#READING IN REDCAP FILE
#reading in the redcap file -- this is just for SIGT to check if SIGT was needed
col_list_redcap = ["record_id", session, "aol_phit_id", "aol_task_id", "datacheck_si_additional"]
df_redcap = pd.read_csv(r'' + redcap_dir + redcap_file,  usecols=col_list_redcap)

#turning the imported excel columns into lists so we can manipulate the data
df_qual_id = df_qual[id_column].tolist()
df_red_id = df_redcap["record_id"].tolist()
df_red_ses_check = df_redcap[session].tolist()
df_phit_id = df_redcap["aol_phit_id"].tolist()
df_task_id = df_redcap["aol_task_id"].tolist()
df_additional_assessments = df_redcap["datacheck_si_additional"].tolist()
            
for x in df_updating_subs:
    loc = df_red_id.index(x) #finding location of sub in redcap file
    if df_additional_assessments[loc]==1:
        if df_red_ses_check[loc]==2: 
            if x in df_qual_id: 
                place_counter_complete=place_counter_complete+1 
                sheet1.write(place_counter_complete, 0, x) 
            if x not in df_qual_id: 
                if df_phit_id[loc] in df_qual_id: #checking if PHIT ID is in qualtrics measure
                    place_counter_complete=place_counter_complete+1 
                    sheet1.write(place_counter_complete, 0, x) 
                if df_phit_id[loc] not in df_qual_id: 
                    if df_task_id[loc] in df_qual_id: #checking if TASK ID is in qualtrics measure
                        place_counter_missing=place_counter_missing+1
                        sheet2.write(place_counter_missing, 0, x)
        
#saving the workborks .xlsx files        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

os.remove(unzip_temp_dir + filename)

NameError: name 'assessment_9_dir' is not defined

In [ ]:
#24. Assessment 10

#setting dir for this measure and identifying subID column in this qualtrics file
measure_dir = assessment_10_dir
id_column = "ID"
session = follow_up_2

#creating the sheets in the excel file where output will be written
sheet1 = wb1.add_sheet('Assessment 10 present')
sheet2 = wb2.add_sheet('Assessment 10 missing')

#putting headers of subID in each excel sheet
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

#SELECTING QUALTRICS FILE
#listing files in measure_dir and selecting top option (there should only be 1 file in each dir -- if not, there could be issues)
dir_file = [f for f in listdir(measure_dir) if isfile(join(measure_dir, f))]
dir_file = dir_file[0]
#extracting the top file from measure_dir and putting into zip_temp_dir
with ZipFile(measure_dir + dir_file, 'r') as zip_ref:
    zip_ref.extractall(unzip_temp_dir)
#listing files in unzip_temp_dir (there should only be 1 file in each dir -- if not, there could be issues)
filename = [f for f in listdir(unzip_temp_dir) if isfile(join(unzip_temp_dir, f))]
#selecting file
filename = filename[0]

#READING IN QUALTRICS FILE
#creating a list of 1 header name. Only this 1 column will be read in
col_list_qual = [id_column]
#reading in the qualtrics file with only the header specified
df_qual = pd.read_csv(r'' + unzip_temp_dir + filename,  usecols=col_list_qual)

#READING IN REDCAP FILE
#reading in the redcap file -- this is just for SIGT to check if SIGT was needed
col_list_redcap = ["record_id", session, "aol_phit_id", "aol_task_id", "datacheck_si_additional"]
df_redcap = pd.read_csv(r'' + redcap_dir + redcap_file,  usecols=col_list_redcap)

#turning the imported excel columns into lists so we can manipulate the data
df_qual_id = df_qual[id_column].tolist()
df_red_id = df_redcap["record_id"].tolist()
df_red_ses_check = df_redcap[session].tolist()
df_phit_id = df_redcap["aol_phit_id"].tolist()
df_task_id = df_redcap["aol_task_id"].tolist()
df_additional_assessments = df_redcap["datacheck_si_additional"].tolist()
            
for x in df_updating_subs:
    loc = df_red_id.index(x) #finding location of sub in redcap file
    if df_additional_assessments[loc]==1:
        if df_red_ses_check[loc]==2: 
            if x in df_qual_id: 
                place_counter_complete=place_counter_complete+1 
                sheet1.write(place_counter_complete, 0, x) 
            if x not in df_qual_id: 
                if df_phit_id[loc] in df_qual_id: #checking if PHIT ID is in qualtrics measure
                    place_counter_complete=place_counter_complete+1 
                    sheet1.write(place_counter_complete, 0, x) 
                if df_phit_id[loc] not in df_qual_id: 
                    if df_task_id[loc] in df_qual_id: #checking if TASK ID is in qualtrics measure
                        place_counter_missing=place_counter_missing+1
                        sheet2.write(place_counter_missing, 0, x)
        
#saving the workborks .xlsx files        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

os.remove(unzip_temp_dir + filename)

In [ ]:
#25. Assessment 11

#setting dir for this measure and identifying subID column in this qualtrics file
measure_dir = assessment_11_dir
id_column = "ID"
session = follow_up_2

#creating the sheets in the excel file where output will be written
sheet1 = wb1.add_sheet('Assessment 11 present')
sheet2 = wb2.add_sheet('Assessment 11 missing')

#putting headers of subID in each excel sheet
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

#SELECTING QUALTRICS FILE
#listing files in measure_dir and selecting top option (there should only be 1 file in each dir -- if not, there could be issues)
dir_file = [f for f in listdir(measure_dir) if isfile(join(measure_dir, f))]
dir_file = dir_file[0]
#extracting the top file from measure_dir and putting into zip_temp_dir
with ZipFile(measure_dir + dir_file, 'r') as zip_ref:
    zip_ref.extractall(unzip_temp_dir)
#listing files in unzip_temp_dir (there should only be 1 file in each dir -- if not, there could be issues)
filename = [f for f in listdir(unzip_temp_dir) if isfile(join(unzip_temp_dir, f))]
#selecting file
filename = filename[0]

#READING IN QUALTRICS FILE
#creating a list of 1 header name. Only this 1 column will be read in
col_list_qual = [id_column]
#reading in the qualtrics file with only the header specified
df_qual = pd.read_csv(r'' + unzip_temp_dir + filename,  usecols=col_list_qual)

#READING IN REDCAP FILE
#reading in the redcap file -- this is just for SIGT to check if SIGT was needed
col_list_redcap = ["record_id", session, "aol_phit_id", "aol_task_id", "datacheck_si_additional"]
df_redcap = pd.read_csv(r'' + redcap_dir + redcap_file,  usecols=col_list_redcap)

#turning the imported excel columns into lists so we can manipulate the data
df_qual_id = df_qual[id_column].tolist()
df_red_id = df_redcap["record_id"].tolist()
df_red_ses_check = df_redcap[session].tolist()
df_phit_id = df_redcap["aol_phit_id"].tolist()
df_task_id = df_redcap["aol_task_id"].tolist()
df_additional_assessments = df_redcap["datacheck_si_additional"].tolist()
            
for x in df_updating_subs:
    loc = df_red_id.index(x) #finding location of sub in redcap file
    if df_additional_assessments[loc]==1:
        if df_red_ses_check[loc]==2: 
            if x in df_qual_id: 
                place_counter_complete=place_counter_complete+1 
                sheet1.write(place_counter_complete, 0, x) 
            if x not in df_qual_id: 
                if df_phit_id[loc] in df_qual_id: #checking if PHIT ID is in qualtrics measure
                    place_counter_complete=place_counter_complete+1 
                    sheet1.write(place_counter_complete, 0, x) 
                if df_phit_id[loc] not in df_qual_id: 
                    if df_task_id[loc] in df_qual_id: #checking if TASK ID is in qualtrics measure
                        place_counter_missing=place_counter_missing+1
                        sheet2.write(place_counter_missing, 0, x)
        
#saving the workborks .xlsx files        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

os.remove(unzip_temp_dir + filename)

In [ ]:
#26. Assessment 12

#setting dir for this measure and identifying subID column in this qualtrics file
measure_dir = assessment_12_dir
id_column = "ID"
session = follow_up_2

#creating the sheets in the excel file where output will be written
sheet1 = wb1.add_sheet('Assessment 12 present')
sheet2 = wb2.add_sheet('Assessment 12 missing')

#putting headers of subID in each excel sheet
sheet1.write(0,0,'subID')
sheet2.write(0,0,'subID')
place_counter_missing=0
place_counter_complete=0

#SELECTING QUALTRICS FILE
#listing files in measure_dir and selecting top option (there should only be 1 file in each dir -- if not, there could be issues)
dir_file = [f for f in listdir(measure_dir) if isfile(join(measure_dir, f))]
dir_file = dir_file[0]
#extracting the top file from measure_dir and putting into zip_temp_dir
with ZipFile(measure_dir + dir_file, 'r') as zip_ref:
    zip_ref.extractall(unzip_temp_dir)
#listing files in unzip_temp_dir (there should only be 1 file in each dir -- if not, there could be issues)
filename = [f for f in listdir(unzip_temp_dir) if isfile(join(unzip_temp_dir, f))]
#selecting file
filename = filename[0]

#READING IN QUALTRICS FILE
#creating a list of 1 header name. Only this 1 column will be read in
col_list_qual = [id_column]
#reading in the qualtrics file with only the header specified
df_qual = pd.read_csv(r'' + unzip_temp_dir + filename,  usecols=col_list_qual)

#READING IN REDCAP FILE
#reading in the redcap file -- this is just for SIGT to check if SIGT was needed
col_list_redcap = ["record_id", session, "aol_phit_id", "aol_task_id", "datacheck_si_additional"]
df_redcap = pd.read_csv(r'' + redcap_dir + redcap_file,  usecols=col_list_redcap)

#turning the imported excel columns into lists so we can manipulate the data
df_qual_id = df_qual[id_column].tolist()
df_red_id = df_redcap["record_id"].tolist()
df_red_ses_check = df_redcap[session].tolist()
df_phit_id = df_redcap["aol_phit_id"].tolist()
df_task_id = df_redcap["aol_task_id"].tolist()
df_additional_assessments = df_redcap["datacheck_si_additional"].tolist()
            
for x in df_updating_subs:
    loc = df_red_id.index(x) #finding location of sub in redcap file
    if df_additional_assessments[loc]==1:
        if df_red_ses_check[loc]==2: 
            if x in df_qual_id: 
                place_counter_complete=place_counter_complete+1 
                sheet1.write(place_counter_complete, 0, x) 
            if x not in df_qual_id: 
                if df_phit_id[loc] in df_qual_id: #checking if PHIT ID is in qualtrics measure
                    place_counter_complete=place_counter_complete+1 
                    sheet1.write(place_counter_complete, 0, x) 
                if df_phit_id[loc] not in df_qual_id: 
                    if df_task_id[loc] in df_qual_id: #checking if TASK ID is in qualtrics measure
                        place_counter_missing=place_counter_missing+1
                        sheet2.write(place_counter_missing, 0, x)
        
#saving the workborks .xlsx files        
wb1.save(save_dir + save_name_present)
wb2.save(save_dir + save_name_missing)

os.remove(unzip_temp_dir + filename)